# Large Circuit Partitioning Benchmark
This notebook builds three composite circuits that combine structured subroutines to exercise QuASAr's partitioning capabilities:

* **Surface-code QAOA hybrid** via `surface_code_qaoa_circuit`, interleaving low-degree QAOA layers with surface-code stabiliser rounds.
* **GHZ–Grover fusion** via `ghz_grover_fusion_circuit`, preparing disjoint GHZ and Grover prefixes before entangling them.
* **QAOA with a Toffoli gadget** via `qaoa_toffoli_gadget_circuit`, inserting a non-Clifford three-qubit gate between MPS-friendly layers.
* **Mixed backend subsystems** via `mixed_backend_subsystems`, combining a Clifford GHZ prefix, an MPS-friendly QAOA mid-block, and a dense random suffix that forces conversions between tableau, MPS, and statevector simulations.

We inspect the resulting partitions, conversion layers, and execution metrics produced by QuASAr.

In [1]:
import os
import sys
from pathlib import Path

os.environ['QUASAR_QUICK_MAX_QUBITS'] = '16'
os.environ['QUASAR_QUICK_MAX_GATES'] = '200'
os.environ['QUASAR_QUICK_MAX_DEPTH'] = '20'

ROOT = Path().resolve().parent.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))


In [2]:
from benchmarks.extensive_circuits import (
    surface_code_qaoa_circuit,
    ghz_grover_fusion_circuit,
    qaoa_toffoli_gadget_circuit,
)
from benchmarks.partition_circuits import mixed_backend_subsystems
from quasar.analyzer import CircuitAnalyzer
from quasar.simulation_engine import SimulationEngine

surface_qaoa = surface_code_qaoa_circuit(bit_width=8, distance=3, rounds=2)
fusion = ghz_grover_fusion_circuit(ghz_qubits=10, grover_qubits=3, iterations=2)
toffoli_gadget = qaoa_toffoli_gadget_circuit(width=18, rounds_before=2, rounds_after=2)

mixed = mixed_backend_subsystems(ghz_width=5, qaoa_width=5, qaoa_layers=2, random_width=4, seed=13)


In [3]:
import time

engine = SimulationEngine()

def _format_subsystems(subsystems):
    if not subsystems:
        return '-'
    return ', '.join('(' + ', '.join(str(q) for q in group) + ')' for group in subsystems)

def describe_partitions(name, partitions):
    if not partitions:
        print(f"[{name}] No partitions identified.")
        return
    for idx, part in enumerate(partitions, start=1):
        backend_name = getattr(part.backend, 'name', str(part.backend))
        history_len = len(part.history)
        subsystem_desc = _format_subsystems(part.subsystems)
        print(
            f"[{name}] Partition {idx}: backend={backend_name}, subsystems=[{subsystem_desc}], gates={history_len}"
        )

def describe_conversions(name, conversions):
    if not conversions:
        print(f"[{name}] No conversion layers.")
        return
    for layer in conversions:
        src = getattr(layer.source, 'name', str(layer.source))
        tgt = getattr(layer.target, 'name', str(layer.target))
        print(
            f"[{name}] Conversion {layer.primitive}: boundary={layer.boundary}, {src}→{tgt}, rank={layer.rank}, frontier={layer.frontier}"
        )

def prepare_circuit(circuit, name):
    print(f"[{name}] Starting analysis...")
    start = time.perf_counter()
    analyzer = CircuitAnalyzer(circuit, estimator=engine.planner.estimator)
    analysis = analyzer.analyze()
    print(f"[{name}] Analysis finished in {time.perf_counter() - start:.2f}s.")
    print(f"[{name}] Preparing execution plan...")
    start = time.perf_counter()
    plan = engine.scheduler.prepare_run(circuit, analysis=analysis)
    duration = time.perf_counter() - start
    total_steps = len(getattr(plan, 'explicit_steps', plan.steps))
    print(f"[{name}] Plan prepared in {duration:.2f}s with {total_steps} step(s).")
    describe_partitions(name, list(circuit.ssd.partitions))
    describe_conversions(name, list(circuit.ssd.conversions))
    return analysis, plan

surface_analysis, surface_plan = prepare_circuit(surface_qaoa, 'Surface-Code QAOA')
fusion_analysis, fusion_plan = prepare_circuit(fusion, 'GHZ–Grover Fusion')
toffoli_analysis, toffoli_plan = prepare_circuit(toffoli_gadget, 'QAOA Toffoli Gadget')

mixed_analysis, mixed_plan = prepare_circuit(mixed, 'Mixed Backend Subsystems')

[Surface-Code QAOA] Starting analysis...
[Surface-Code QAOA] Analysis finished in 0.00s.
[Surface-Code QAOA] Preparing execution plan...


[Surface-Code QAOA] Plan prepared in 1.87s with 1 step(s).
[Surface-Code QAOA] Partition 1: backend=STATEVECTOR, subsystems=[(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20)], gates=128
[Surface-Code QAOA] No conversion layers.
[GHZ–Grover Fusion] Starting analysis...
[GHZ–Grover Fusion] Analysis finished in 0.00s.
[GHZ–Grover Fusion] Preparing execution plan...


[GHZ–Grover Fusion] Plan prepared in 1.03s with 1 step(s).
[GHZ–Grover Fusion] Partition 1: backend=MPS, subsystems=[(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)], gates=106
[GHZ–Grover Fusion] No conversion layers.
[QAOA Toffoli Gadget] Starting analysis...
[QAOA Toffoli Gadget] Analysis finished in 0.01s.
[QAOA Toffoli Gadget] Preparing execution plan...
[QAOA Toffoli Gadget] Plan prepared in 0.01s with 1 step(s).
[QAOA Toffoli Gadget] Partition 1: backend=STATEVECTOR, subsystems=[(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17)], gates=177
[QAOA Toffoli Gadget] No conversion layers.


In [4]:
from itertools import count

def simulate_prepared(circuit, plan, analysis, name):
    total_steps = len(getattr(plan, 'explicit_steps', plan.steps))
    step_counter = count(1)

    def monitor(step, observed, estimated):
        idx = next(step_counter)
        backend_name = getattr(step.backend, 'name', str(step.backend))
        gate_count = step.end - step.start
        est_time = getattr(estimated, 'time', 0.0)
        obs_time = getattr(observed, 'time', 0.0)
        print(
            f"[{name}] Step {idx}/{total_steps} on {backend_name}: {gate_count} gate(s), est. {est_time:.2f}s, observed {obs_time:.2f}s."
        )

    print(f"[{name}] Executing prepared plan with {total_steps} step(s)...")
    conversions = list(getattr(plan, 'explicit_conversions', ()))
    if conversions:
        print(f"[{name}] Planned conversions:")
        describe_conversions(name, conversions)
    ssd, metrics = engine.scheduler.run(
        circuit,
        plan,
        analysis=analysis,
        instrument=True,
        monitor=monitor,
    )
    runtime = metrics.cost.time
    peak_memory = (metrics.cost.memory / (1024 ** 3)) if metrics.cost.memory else 0.0
    print(
        f"[{name}] Simulation complete in {runtime:.2f}s with peak memory {peak_memory:.2f} GiB and {metrics.backend_switches} backend switch(es)."
    )
    print(f"[{name}] Final SSD partitions: {len(ssd.partitions)}")
    return ssd, metrics

surface_ssd, surface_metrics = simulate_prepared(surface_qaoa, surface_plan, surface_analysis, 'Surface-Code QAOA')
fusion_ssd, fusion_metrics = simulate_prepared(fusion, fusion_plan, fusion_analysis, 'GHZ–Grover Fusion')
toffoli_ssd, toffoli_metrics = simulate_prepared(toffoli_gadget, toffoli_plan, toffoli_analysis, 'QAOA Toffoli Gadget')

mixed_ssd, mixed_metrics = simulate_prepared(mixed, mixed_plan, mixed_analysis, 'Mixed Backend Subsystems')

[Surface-Code QAOA] Executing prepared plan with 1 step(s)...
[Surface-Code QAOA] Step 1/1 on STATEVECTOR: 128 gate(s), est. 387.14s, observed 0.02s.


[Surface-Code QAOA] Simulation complete in 0.02s with peak memory 0.03 GiB and 0 backend switch(es).
[Surface-Code QAOA] Final SSD partitions: 1
[GHZ–Grover Fusion] Executing prepared plan with 1 step(s)...
[GHZ–Grover Fusion] Step 1/1 on MPS: 106 gate(s), est. 7798857.65s, observed 0.01s.
[GHZ–Grover Fusion] Simulation complete in 0.01s with peak memory 0.00 GiB and 0 backend switch(es).
[GHZ–Grover Fusion] Final SSD partitions: 1
[QAOA Toffoli Gadget] Executing prepared plan with 1 step(s)...
[QAOA Toffoli Gadget] Step 1/1 on STATEVECTOR: 177 gate(s), est. 46399488.01s, observed 0.01s.
[QAOA Toffoli Gadget] Simulation complete in 0.01s with peak memory 0.00 GiB and 0 backend switch(es).
[QAOA Toffoli Gadget] Final SSD partitions: 1
